In [29]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [30]:
def data_filter(dat):
    dat['date'] = dat['date'].apply(lambda x: x.split('+')[0])
    dat['date'] = pd.to_datetime(dat['date']) # converting the date column to datetime format
    dat = dat[dat['date'] > '2017-12-31' ]
    dat = dat[dat['date'] < '2021-08-01' ]
    return dat

def is_body_size_greater_than_nx_average(df, n, window):
    df['candle_body'] = abs(df['close'] - df['open'])
    df['avg'] = df['candle_body'].rolling(window=window).mean()

    return (df['candle_body'] > df['avg']*n) & (df['open']>df['close'])

def add_imbalance_zone(dat):
#     dat['upper1'] = np.where(dat['is_imbalance'] , (min(dat['min_last_30'] , dat['open'])) , 0)
#     dat['lower1'] = np.where(dat['is_imbalance'], dat['high'].shift(-1), 0)
    
    dat.loc[dat["is_imbalance"], "upper"] = np.minimum( dat.loc[dat["is_imbalance"], "min_last_30"], dat.loc[dat["is_imbalance"], "open"])  
    dat.loc[dat["is_imbalance"], "lower"] = dat.shift(-1).loc[dat["is_imbalance"], "high"]
    
    return dat

In [31]:
dat = pd.read_csv('ACC_minute_data_with_indicators.csv', usecols = ['date' , 'open' , 'high', 'low' , 'close' , 'volume'] )
dat

,date,close,high,low,open,volume
0,2015-02-02 10:18:00+05:30,1544.00,1545.00,1543.10,1545.00,220
1,2015-02-02 10:19:00+05:30,1543.70,1544.00,1543.00,1544.00,29
2,2015-02-02 10:20:00+05:30,1543.00,1543.70,1542.50,1543.70,28
3,2015-02-02 10:21:00+05:30,1542.30,1543.00,1542.30,1543.00,125
4,2015-02-02 10:22:00+05:30,1542.10,1542.15,1541.10,1541.10,113
...,...,...,...,...,...,...
660528,2022-10-24 19:10:00+05:30,2255.00,2255.90,2254.55,2255.80,199
660529,2022-10-24 19:11:00+05:30,2255.95,2255.95,2254.55,2255.75,227
660530,2022-10-24 19:12:00+05:30,2256.25,2256.25,2255.35,2256.00,594
660531,2022-10-24 19:13:00+05:30,2255.00,2255.95,2254.55,2255.50,267


In [32]:
dat = data_filter(dat)

In [33]:
dat.head()

,date,close,high,low,open,volume
249971,2018-01-01 09:15:00,1760.00,1763.90,1758.65,1763.90,718
249972,2018-01-01 09:16:00,1763.80,1763.80,1760.00,1760.00,409
249973,2018-01-01 09:17:00,1760.10,1763.45,1760.00,1763.45,867
249974,2018-01-01 09:18:00,1759.35,1760.45,1759.35,1760.10,1065
249975,2018-01-01 09:19:00,1758.65,1759.90,1758.65,1759.35,436


In [34]:
dat.shift(-1).head()

,date,close,high,low,open,volume
249971,2018-01-01 09:16:00,1763.80,1763.80,1760.00,1760.00,409.0
249972,2018-01-01 09:17:00,1760.10,1763.45,1760.00,1763.45,867.0
249973,2018-01-01 09:18:00,1759.35,1760.45,1759.35,1760.10,1065.0
249974,2018-01-01 09:19:00,1758.65,1759.90,1758.65,1759.35,436.0
249975,2018-01-01 09:20:00,1758.50,1761.00,1758.50,1758.65,591.0


In [35]:
dat['is_imbalance'] = is_body_size_greater_than_nx_average(dat, 3, 30)

In [36]:
dat['is_imbalance'].sum()

6420

In [37]:
dat['min_last_30'] = dat['low'].shift(1).rolling(window=30).min()
dat['min_last_50'] = dat['low'].shift(1).rolling(window=30).min()

In [38]:
dat.tail()

,date,close,high,low,open,volume,candle_body,avg,is_imbalance,min_last_30,min_last_50
554670,2021-07-28 15:25:00,2391.70,2396.75,2390.15,2395.35,2622,3.65,0.901667,True,2388.5,2388.5
554671,2021-07-28 15:26:00,2395.00,2395.35,2391.70,2391.70,2092,3.30,0.995000,False,2388.5,2388.5
554672,2021-07-28 15:27:00,2391.15,2395.00,2391.15,2394.95,1189,3.80,1.081667,True,2388.5,2388.5
554673,2021-07-28 15:28:00,2386.10,2394.60,2384.80,2391.15,3110,5.05,1.250000,True,2388.5,2388.5
554674,2021-07-28 15:29:00,2392.00,2394.00,2386.00,2386.10,846,5.90,1.418333,False,2384.8,2384.8


In [39]:
dat[dat["is_imbalance"]]

,date,close,high,low,open,volume,candle_body,avg,is_imbalance,min_last_30,min_last_50
250005,2018-01-01 09:49:00,1747.25,1750.35,1747.25,1750.35,157,3.10,1.006667,True,1746.70,1746.70
250047,2018-01-01 10:31:00,1748.55,1751.25,1747.00,1751.25,705,2.70,0.851667,True,1743.35,1743.35
250128,2018-01-01 11:52:00,1750.00,1751.75,1750.00,1751.75,76,1.75,0.368333,True,1750.00,1750.00
250138,2018-01-01 12:02:00,1750.00,1751.10,1750.00,1751.10,96,1.10,0.335000,True,1750.00,1750.00
250142,2018-01-01 12:06:00,1749.00,1750.30,1748.40,1750.30,141,1.30,0.375000,True,1750.00,1750.00
250143,2018-01-01 12:07:00,1746.85,1749.00,1746.00,1749.00,689,2.15,0.421667,True,1748.40,1748.40
250148,2018-01-01 12:12:00,1746.40,1747.90,1746.40,1747.90,170,1.50,0.485000,True,1746.00,1746.00
250204,2018-01-01 13:08:00,1746.00,1748.50,1746.00,1748.50,431,2.50,0.726667,True,1748.00,1748.00
250260,2018-01-01 14:04:00,1745.00,1746.10,1745.00,1746.10,152,1.10,0.300000,True,1743.60,1743.60
250273,2018-01-01 14:17:00,1746.05,1747.10,1745.70,1747.10,170,1.05,0.221667,True,1744.60,1744.60


In [42]:
dat = add_imbalance_zone(dat)

In [43]:
dat["zone"] = (dat["upper"] - dat["lower"])>0

In [56]:
xx = list((dat[dat["zone"]]["date"].apply(lambda x: str(x))))#.sum()
import pyperclip
pyperclip.copy(str(xx))

In [ ]:
df2 = dat
df2 =  dat.copy()
pd.options.display.max_rows = len(df2)

In [ ]:
df2["lows"]=df2["lower"]*(df2["zone"].astype(int))
df2["lows"] = df2["lows"].where(df2["lows"]!=0,np.nan)
df2["lows"].fillna(method = "ffill",inplace = True)
df2["idxlow"]=df2.index*(df2["zone"].astype(int))
df2["idxlow"] = df2["idxlow"].where(df2["idxlow"]!=0,np.nan)
df2["idxlow"].fillna(method = "ffill",inplace = True)
df2["cond"] = (df2["close"]>df2["lows"]).astype(int)

print(df2.loc[2302:2310])
df2["cond"] = df2["cond"].where(df2["cond"]!=0,np.nan)
print(df2.loc[2302:2310])

df2["fzon"] = df2["zone"].astype(int)==0
df2["cond"][df2["zone"]] = df2[df2["zone"]]["fzon"]
print(df2.loc[2302:2310])
df2["cond"].fillna(method = "ffill",inplace = True)
df2["cond"].fillna(method = "bfill",inplace = True)
print(df2.loc[2302:2310])
df2["ii"]=df2["cond"].astype(int)
df2["firstTrue"] = (df2["ii"]-df2["ii"].shift(1))>0
df2["count"] = np.nan
df2["count"][df2["firstTrue"]] =  df2["firstTrue"][df2["firstTrue"]].index - df2["idxlow"][df2["firstTrue"]]
df2["count"].fillna(method = "bfill",inplace = True)
df2["count"] = df2["count"]*(df2["zone"].astype(int))

In [25]:
df2["count"].value_counts()

0.0      302356
2.0         660
3.0         316
4.0         214
5.0         122
6.0          83
7.0          60
8.0          57
9.0          54
10.0         34
12.0         30
11.0         28
13.0         27
17.0         21
19.0         19
15.0         18
14.0         14
20.0         14
16.0         13
23.0         11
18.0         11
22.0         10
27.0          9
28.0          9
38.0          7
21.0          7
25.0          7
31.0          6
26.0          6
29.0          6
77.0          5
34.0          5
32.0          5
45.0          5
39.0          4
69.0          4
43.0          3
58.0          3
24.0          3
33.0          3
79.0          3
74.0          3
47.0          3
30.0          3
55.0          2
140.0         2
53.0          2
49.0          2
40.0          2
109.0         2
80.0          2
42.0          2
35.0          2
94.0          2
70.0          2
48.0          1
57.0          1
549.0         1
62.0          1
100.0         1
88.0          1
131.0         1
65.0    

In [120]:
!pip install pyperclip

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11113 sha256=7e6b601bf278a876a682dbc698ec14cee60dc80f0f204f5000fadf30ab683e3a
  Stored in directory: c:\users\hksci\appdata\local\pip\cache\wheels\9f\18\84\8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


You should consider upgrading via the 'c:\users\hksci\anaconda3\python.exe -m pip install --upgrade pip' command.


In [125]:
pd.options.display.max_rows = len(df2)

In [138]:
(df2["count"].value_counts().keys()).max()

549.0

In [34]:
dat.to_csv("test1.csv")

In [40]:
df = pd.DataFrame()
df

""


In [ ]:
def stats(dat):
    value_counts = dat["retraced_days"].value_counts()
    

In [35]:
files = pd.read_csv("Untitled spreadsheet - Sheet1.csv")
files

AttributeError: module 'pandas' has no attribute 'Dataframe'

In [ ]:
for file in files:
    dat = pd.read_csv(file, usecols = ['date' , 'open' , 'high', 'low' , 'close' , 'volume'])
    dat = data_filter(dat)
    dat['is_imbalance'] = is_body_size_greater_than_nx_average(dat, 3, 30)
    dat['min_last_30'] = dat['low'].shift(1).rolling(window=30).min()
    add_imbalance_zone(dat)
    dat["zone"] = (dat["upper"] - dat["lower"])>0
    dat = retraced_days(dat)
    df["file_name"] = 
    